<p style="text-align:center"><img src="Julia.jpg" width="350" ></p>

# I/O Input / Output
[web de donde leer este tema](https://docs.julialang.org/en/v1/base/io-network/)

## Formato de datos Julia - JLD2

`JLD2` guarda y carga estructuras de datos de `Julia` en un formato que comprende un subconjunto de `HDF5`, sin ninguna dependencia de la biblioteca `C` de `HDF5`. 

## HDF5.jl
#### Visión general

`HDF5` significa **H**ierarchical **D**ata **F**ormat **v5** y está estrechamente modelado en los sistemas de archivos.  
En `HDF5`, un "grupo" es análogo a un directorio, un "conjunto de datos" es como un archivo.  
`HDF5` también usa "atributos" para asociar metadatos con un grupo o conjunto de datos en particular.  
`HDF5` usa nombres `ASCII` para estos diferentes objetos, y se puede acceder a los objetos mediante nombres de ruta similares a Unix, por ejemplo, "/sample1/tempsensor/firsttrial" para un grupo de nivel superior "sample1", un subgrupo "tempsensor" y un conjunto de datos "firsttrial". 

Mientras que otras implementaciones de HDF5 que admiten la especificación de formato de archivo `HDF5 versión 3.0` (es decir, libhdf5 1.10 o posterior) deberían poder leer los archivos que produce `JLD2`, es probable que `JLD2` no pueda leer archivos creados o modificados por otras implementaciones de `HDF5`.  
`JLD2` no pretende ser compatible hacia atrás o hacia adelante con el paquete `JLD`. 

### Leer y escribir datos  
Una nueva interfaz: `jldsave`

`jldsave`hace uso de la sintaxis de argumento de palabra clave de `Julia` para almacenar archivos, aprovechando así el analizador y no teniendo que depender de macros.  
Para usarlo se escribe:

### Installation

Within Julia, use the package manager:

In [1]:
# import Pkg; Pkg.add("JLD2")

In [2]:
# To use the JLD2 module, begin your code with
using JLD2

In [3]:
x = 17

17

In [4]:
save("example_1.jld2", Dict("x" => x)) # Aqui ponemos "x" -> x, en un diccionario

In [5]:
z = load("example_1.jld2") # cargamos todo en 'z'

Dict{String, Any} with 1 entry:
  "x" => 17

In [6]:
print(z)  # Podemos imprimir para mostrar los datos

Dict{String, Any}("x" => 17)

-------------------------------------
-------------------------------------
### `jldsave`
Con la función `jldsave` grabo directamente en el disco

Probemos con más datos!

In [7]:
x = 1
y = 2
z = 42
t = [1,3]
print(x,' ', y,' ', z,' ', t)

1 2 42 [1, 3]

Ahora los guardo con `jldsave`

In [8]:
# The simplest case:
jldsave("example_2.jld2"; x, y, z, t)
# it is equivalent to 
#  jldsave("example.jld2"; x=x, y=y, z=z)

# You can assign new names selectively
#  jldsave("example.jld2"; x, a=y, z)

# and if you want to confuse your future self and everyone else, do
#  jldsave("example.jld2"; z=x, x=y, y=z)

In [9]:
x = nothing
y = nothing
z = nothing
t = nothing

In [10]:
print(x,' ',y,' ', z,' ', t)

nothing nothing nothing nothing

-------------------------------------
-------------------------------------
### `jldopen`
Ahora abrimos el archivo y cargamos los datos con `jldopen`.

In [11]:
f = jldopen("example_2.jld2", "r")  # open read-only (default)
close(f)

Ahora los mostramos

In [12]:
print(f)   # Ups, no se ve nada

JLDFile /home/edgardo/Documents/Julia/curso-julia-2022-1-priv/clases/08-guardar-archivos/example_2.jld2 (read-only)

In [13]:
println(f["t"]) # Ups, aqui tampoco! que pasó?

LoadError: ArgumentError: file is closed

sucede que lo habíamos cerrado, lo hagamos de nuevo, pero mostremos los datos antes de cerrar!

In [14]:
f = jldopen("example_2.jld2", "r")  # open read-only (default)
println(f["t"])
println(f["x"])
println(f["y"])
println(f["z"])
close(f)

[1, 3]
1
2
42


------------------------------
------------------------------
Dado que al archivo ya lo cerré, me dará un error cuando lo quiere imprimir

In [15]:
println(f["t"])  

LoadError: ArgumentError: file is closed

------------------------------------------
------------------------------------------
## `load`  
Ahora usaremos el comando `load`.   
En este caso lo estoy cargando en memoria, y perdurará

In [16]:
file = load("example_2.jld2")

Dict{String, Any} with 4 entries:
  "t" => [1, 3]
  "x" => 1
  "z" => 42
  "y" => 2

In [17]:
Te = file["t"]

2-element Vector{Int64}:
 1
 3

In [18]:
print(Te)

[1, 3]

probemos de cerrarlo, a ver que sucede!

In [19]:
close(file)  # No hay método para cerrar, cuando lo abrimos de esta forma.

LoadError: MethodError: no method matching close(::Dict{String, Any})
[0mClosest candidates are:
[0m  close([91m::Union{Base.AsyncCondition, Timer}[39m) at asyncevent.jl:136
[0m  close([91m::Union{FileWatching.FileMonitor, FileWatching.FolderMonitor, FileWatching.PollingFileWatcher}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/FileWatching/src/FileWatching.jl:304
[0m  close([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m) at /home/edgardo/.julia/packages/ZMQ/R3wSD/src/_message.jl:25
[0m  ...

-------------------------
-------------------------
Y los datos quedan almacenados en `file`, probemos...

In [20]:
Zeta = file["z"]

42

-------------------------------
-------------------------------
Escribamos la siguiente estructura

In [21]:
file = jldopen("example_2.jld2", "a+")  # Lo abre en el disco para agregar elementos
file["w/visus"] = "total"
# close(z)

"total"

In [22]:
print(file["w/visus"])

total

In [23]:
close(file)

Si hacemos load, no tenemos que usar el `close`

In [24]:
file = load("example_2.jld2") # Lo carga en memoria
print(file["z"])
#close(file)

42

No obstante, si hacemos `jldopen` en ese caso `si` debemos hacer `close`

In [25]:
f = jldopen("example_2.jld2", "r")  # open read-only (default)
#f = jldopen("example_2.jld2", "r+") # open read/write, failing if no file exists
#f = jldopen("example_2.jld2", "w")  # open read/write, overwriting existing file
#f = jldopen("example_2.jld2", "a+") # open read/write, preserving contents of existing file or creating a new file
close(f) # Siempre que abramos con jldopen debemos cerrar para seguir operando

In [26]:
file = jldopen("example_2.jld2", "r")  # Aquí lo abrimos con `r `

JLDFile /home/edgardo/Documents/Julia/curso-julia-2022-1-priv/clases/08-guardar-archivos/example_2.jld2 (read-only)
 ├─🔢 x
 ├─🔢 y
 ├─🔢 z
 ├─🔢 t
 └─📂 w
    └─🔢 visus

In [27]:
println(file["z"])
println(file["y"])
println(file["x"])

42
2
1


In [28]:
close(file)

### Veamos un ejemplo para agregar varios datos

``` Julia
jldopen("example_3.jld2", "a") do file
    file["bigdata"] = randn(10)  # Usamos random para generar cualquier número
end
```

In [29]:
jldopen("example_3.jld2", "a+") do file
    file["bigdata"] = randn(10) 
    file["mediumdata"] = randn(5)     
    file["smalldata"] = randn(2) # put your data here instead of random
end

2-element Vector{Float64}:
 -0.2838928307032815
 -0.5076908053773049

--------------------------
Agregamos mas datos!

In [30]:
jldopen("example_3.jld2", "a+") do file
    file["toto"] = "fofo" 
end

"fofo"

-------------------------------
miremos como el estatus `w+` sobre escribirá el archivo

In [40]:
jldopen("example_3.jld2", "w+") do file
    file["big"] = randn(8) 
    file["small"] = randn(2) # put your data here instead of random
    file["tata"] = "fafa" 
end

"fafa"

leamos los datos

In [41]:
file = jldopen("example_3.jld2", "r")

JLDFile /home/edgardo/Documents/Julia/curso-julia-2022-1-priv/clases/08-guardar-archivos/example_3.jld2 (read-only)
 ├─🔢 big
 ├─🔢 small
 └─🔢 tata

In [33]:
close(file) # Recordemos cerrar el archivo!

-------------------------
Hagamos un último ejemplo

Aquí uso `w+`

In [52]:
jldopen("example_4.jld2", "w+") do file
    file["big"] = randn(8) 
    file["small"] = randn(2) # put your data here instead of random
end   # Obsrvemos que con este `end` estoy cerrando el archivo

2-element Vector{Float64}:
 1.3349483464672027
 0.15372692262200915

----------------------------
veamos que guardó

In [53]:
archivo = jldopen("example_4.jld2", "r")

JLDFile /home/edgardo/Documents/Julia/curso-julia-2022-1-priv/clases/08-guardar-archivos/example_4.jld2 (read-only)
 ├─🔢 big
 └─🔢 small

Recordemos cerrar el archivo!

In [54]:
close(archivo)

-------------------------
Agreguemos más datos, con `a`

In [55]:
jldopen("example_4.jld2", "a") do file
    file["bigdata"] = randn(8) 
    file["smalldata"] = randn(2) # put your data here instead of random
    file["tata"] = "fafa" 
end

"fafa"

y veamos que tiene ahora

In [56]:
archivo = jldopen("example_4.jld2", "r")

JLDFile /home/edgardo/Documents/Julia/curso-julia-2022-1-priv/clases/08-guardar-archivos/example_4.jld2 (read-only)
 ├─🔢 big
 ├─🔢 small
 ├─🔢 bigdata
 ├─🔢 smalldata
 └─🔢 tata